In [1]:
import foolbox as fb
import art
import tensorflow as tf
import numpy as np
import joblib

/home/bossantm/anaconda3/envs/SDNIOT-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bossantm/anaconda3/envs/SDNIOT-env/lib/python3.10/site-packages/art/estimators/certification/__init__.py:30: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")
2025-06-18 16:22:34.868078: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
X_train = joblib.load('../../preprocessed_data/InSDN/X_train.joblib')
X_test = joblib.load('../../preprocessed_data/InSDN/X_test.joblib')
Y_train = joblib.load('../../preprocessed_data/InSDN/Y_train.joblib')
Y_test = joblib.load('../../preprocessed_data/InSDN/Y_test.joblib')

print (X_test[:10])
print (Y_test[:10])

[[[-3.08062434e-01]
  [-3.91238695e-03]
  [-3.79172899e-02]
  [-1.41024180e-02]
  [-2.39897799e-02]
  [-1.71919852e-01]
  [-1.44635454e-01]
  [-1.62529573e-01]
  [-1.51525512e-01]
  [-1.42814651e-01]
  [-2.44456157e-01]
  [-2.46024951e-01]
  [-2.28204712e-01]
  [-1.11989252e-01]
  [ 3.09812450e+00]
  [-2.69014388e-01]
  [-2.90514976e-01]
  [-3.00209582e-01]
  [-2.02588961e-02]
  [-1.78631395e-01]
  [-1.40319541e-01]
  [-1.45762533e-01]
  [-1.64758474e-01]
  [-3.51814628e-02]
  [-3.01634163e-01]
  [-2.68204480e-01]
  [-2.83928454e-01]
  [-2.96158254e-01]
  [-2.02220958e-02]
  [-2.38267884e-01]
  [-1.09555945e-01]
  [-6.67901039e-02]
  [-5.71007393e-02]
  [-6.40785694e-02]
  [ 3.11412477e+00]
  [-2.88815498e-01]
  [-1.57558277e-01]
  [-2.78688818e-01]
  [-2.41472840e-01]
  [-4.35938314e-02]
  [-3.99482548e-01]
  [-5.57206988e-01]
  [-3.35715450e-02]
  [-2.38267884e-01]
  [-6.14396274e-01]
  [-1.09555945e-01]
  [-6.66714251e-01]
  [-2.92166054e-01]
  [-1.62529573e-01]
  [-2.46024951e-01]


In [3]:
from tensorflow.keras.models import load_model

cnn_loaded = load_model('../../models/cnn_insdn_model.keras')
lstm_loaded = load_model('../../models/lstm_insdn_model.keras')
ae_lstm_loaded = load_model('../../models/ae-lstm_insdn_model.keras')

In [4]:
from art.attacks.evasion import FeatureAdversariesTensorFlowV2, HopSkipJump, BoundaryAttack
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_mnist
from tensorflow.keras.utils import to_categorical

classifier = TensorFlowV2Classifier(
    model=ae_lstm_loaded,
    nb_classes=2,
    input_shape=(X_test.shape[1], X_test.shape[2]),
    loss_object=tf.keras.losses.BinaryCrossentropy(),
    clip_values=(0.0, 1.0)
)


# si Y_test est de forme (n,) avec 0 ou 1 :
Y_test_categorical = to_categorical(Y_test, num_classes=2)


attack_hsj = HopSkipJump(classifier=classifier)
adv_hopskipjump = attack_hsj.generate(x=X_test[:10])
attack_boundary = BoundaryAttack(estimator=classifier)
adv_boundary = attack_boundary.generate(x=X_test[:10], y=Y_test[:10])

# original predictions
preds_orig = classifier.predict(X_test[:10])

# HopSkipJump predictions
preds_adv_hsj = classifier.predict(adv_hopskipjump)

# BoundaryAttack predictions
preds_adv_boundary = classifier.predict(adv_boundary)

print("Original predictions       :", np.argmax(preds_orig, axis=1))
print("After HopSkipJump attack   :", np.argmax(preds_adv_hsj, axis=1))
print("After BoundaryAttack       :", np.argmax(preds_adv_boundary, axis=1))

Boundary attack: 100%|██████████| 10/10 [00:18<00:00,  1.83s/it]

Original predictions       : [0 0 0 0 0 0 0 0 0 0]
After HopSkipJump attack   : [0 0 0 0 0 0 0 0 0 0]
After BoundaryAttack       : [0 0 0 0 0 0 0 0 0 0]
